In [1]:
import torch
from torch import tanh
from torch import nn
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
text_input = '/home/duyongkang/PaperRec/Data/Glove.pkl'

In [2]:
data_t = pd.read_pickle(text_input)

In [3]:
t_dic = data_t[['node','vector']].set_index('node').to_dict()['vector']

In [4]:
from numpy import multiply
info = pd.read_csv('/home/duyongkang/aminer/aminer.author.paper.link.csv')
ppr_list = info['paper_id'].values.tolist()
auth_list = info['author_id'].values.tolist()

In [5]:
ppr = []
auth = []
for i in range(len(ppr_list)):
    p = 'p' + str(ppr_list[i])
    a = 'a' + str(auth_list[i])
    if p in t_dic.keys() and a in t_dic.keys():
        ppr.append(p)
        auth.append(a)
    else:
        continue

In [6]:
from pandas.core.frame import DataFrame
data_dic = {"author":auth, "paper":ppr}
dataset = DataFrame(data_dic)

In [7]:
# 统计每篇文章的作者，key为论文标号，value为作者列表
pa_dic = {}
for i in range(len(ppr_list)):
    p = 'p' + str(ppr_list[i])
    a = 'a' + str(auth_list[i])
    try:
        pa_dic[p].append(a)
    except KeyError:
        pa_dic[p] = []
        pa_dic[p].append(a)

# 统计每个作者的文章，key为作者编号，value为论文列表
ap_dic = {}
for i in range(len(auth_list)):
    p = 'p' + str(ppr_list[i])
    a = 'a' + str(auth_list[i])
    try:
        ap_dic[a].append(p)
    except KeyError:
        ap_dic[a] = []
        ap_dic[a].append(p)

In [8]:
df_train=dataset[:int(0.8*len(dataset))]
df_train['label'] = 1
df_train

/home/duyongkang/.conda/envs/gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,author,paper,label
0,a678283,p782,1
1,a644976,p872,1
2,a107969,p872,1
3,a16959,p977,1
4,a500399,p1014,1
...,...,...,...
81531,a19813,p1728161,1
81532,a782515,p1728161,1
81533,a1456526,p1728161,1
81534,a688,p1728161,1


In [9]:
import random
mul = []
lenth = len(df_train)
for i in range(lenth):
    a = df_train.author[i]
    while(1):
        # 生成随机论文
        p = dataset.paper[random.randint(0,len(dataset)-1)]
        #判断随机找到的论文是否在该作者的论文列表中
        if p in ap_dic[a] :
            continue
        else:
            break
    #将随机作者和该论文向量点乘
    new = pd.DataFrame({'author':a,'paper':p,'label':0},index=[1])
    df_train = df_train.append(new,ignore_index=True)   # ignore_index=True,表示不按原来的索引，从0开始自动递增

In [ ]:
mul = []
for i in range(len(df_train)):
    a = df_train.author[i]
    p = df_train.paper[i]
    v = multiply(t_dic[a],t_dic[p])
    mul.append(v)

df_train['vector'] = mul

## 测试集

In [12]:
df_test=dataset[int(0.8*len(dataset) ):]
df_test = df_test.sort_values(by="author" , ascending=False) #by 指定列 ascending
df_test = df_test.reset_index(drop=True)
df_test['label'] = 1

In [15]:
count = {}
ap_test = {}
for i in range(len(df_test)):    
    try:
        count[df_test.author[i]]+=1
        ap_test[df_test.author[i]].append(df_test.paper[i])
    except:
        count[df_test.author[i]] = 0
        ap_test[df_test.author[i]] = []

## 添加负样本

In [ ]:
import copy
info = pd.read_csv('/home/duyongkang/aminer/aminer.paper.info.csv')
ppr_list = info['paper_id'].values.tolist()             # 论文列表
text_list = info['text'].values.tolist()                # 文本列表
cit_list = info['citations'].values.tolist()            #引用列表
confer_list = info['conference'].values.tolist()        #会议列表

# 论文的引用
cit_dic = {}      # key为论文编号，value为引用论文编号
for i in range(len(ppr_list)):
    ppr = 'p' + str(ppr_list[i])
    cit_dic[ppr] = []
    lenth = len(cit_list[i])
    if lenth == 1:
        continue
    elif lenth > 1 and lenth < 8:
        cit = 'p' + str(cit_list[i])
        cit_dic[ppr].append(cit)
    elif lenth > 8:
        cit = cit_list[i].split(';')
        cit = ['p'+num for num in cit]
        cit_dic[ppr].extend(cit)
        
def Cit_Paper( paper ):
    return cit_dic[paper]

# 同一个会议的论文
pc_dic = dict(zip(ppr_list,confer_list))              #key为论文，value为会议
confer_dic = {}
for i in range(len(ppr_list)):
    confer = confer_list[i]
    ppr = 'p' + str(ppr_list[i])
    if confer not in confer_dic.keys():
        confer_dic[confer] = []
    confer_dic[confer].append(ppr)
    
def Same_Confer_Paper( paper ):
    confer = pc_dic[int(paper[1:])]
    temp = copy.deepcopy(confer_dic[confer])
    temp.remove(paper)
    return temp

## 按引用和会议添加负样本

In [ ]:
i = 0
for author in ap_test.keys():
    i += 1
    full = False
    for paper in ap_test[author]:
        if full:
            break
        # 找该论文引用的论文，加到负样本中
        for cit_p in Cit_Paper(paper):
            if cit_p in ap_test[author]:
                continue
            try:
                new = pd.DataFrame({'author':author,'paper':cit_p,'label':0},index=[1])
                df_test = df_test.append(new,ignore_index=True)
                count[author]+=1
                if(count[author] == 50):
                    full = True
                    break
            except:
                continue
        # 如果样本不够，再找同一个会议的论文，加入样本中
        if not full:
            for same_confer_p in Same_Confer_Paper(paper):
                if same_confer_p in ap_test[author]:
                    continue
                try:
                    new = pd.DataFrame({'author':author,'paper':same_confer_p,'label':0},index=[1])
                    df_test = df_test.append(new,ignore_index=True)
                    count[author]+=1
                    if(count[author] == 50):
                        full = True
                        break
                except:
                    continue
    if i % 100 == 0:
        print(i)

## 随机添加负样本

In [16]:
import random
i = 0
for a in count:
    while(count[a] < 50):
        temp = []
        while(1):
            p = dataset.paper[random.randint(0,len(dataset)-1)]
            if p in ap_dic[a] or p in temp:
                continue
            else:
                temp.append(p)
                break
        v = [multiply(t_dic[a],t_dic[p])]
        new = pd.DataFrame({'author':a,'paper':p,'vector':v,'label':0},index=[1])
        df_test = df_test.append(new,ignore_index=True)
        count[a]+=1
    i += 1
    if(i%1000==0):
        print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


## 计算点积

In [13]:
a = df_test.author.to_list()
p = df_test.paper.to_list()
mul = []
for i in range(len(df_test)):
    a = df_test.author[i]
    p = df_test.paper[i]
    v = multiply(t_dic[a],t_dic[p])
    mul.append(v)

df_test['vector'] = mul

In [17]:
df_test = df_test.sort_values(by="author" , ascending=False) #by 指定列 ascending
df_test = df_test.reset_index(drop=True)

In [18]:
# 将tensor转换成list
def to_list(y):
    y = y.cpu()
    try:
        y = y.numpy()
    except:
        y = y.detach().numpy()
    y = y.astype(np.float).tolist()
    y = [y[i][0] for i in range(len(y))]
    return y
def HR_score(y_true, y_pred, k, truncate=False):
    n_positive = y_true.tolist().count(1)
    # make sure y_true and y_pred have the same length
    assert len(y_true) == len(y_pred)
    assert k > 0 and n_positive > 0

    # truncate y_true and y_pred according to k and pos num
    if truncate:
        topk = min(k, y_true.tolist().count(1))
    else:
        topk = k
    order = np.argsort(y_pred, kind="stable")[-topk:][::-1]
    y_true = y_true[order]

    return (y_true > 0).sum() / n_positive

def DCG_score(y_true, y_pred, k, truncate=False):
    # make sure y_true and y_pred have the same length
    assert len(y_true) == len(y_pred)
    assert k > 0 and y_true.tolist().count(1) > 0

    # truncate y_true and y_pred according to k and pos num
    if truncate:
        topk = min(k, y_true.tolist().count(1))
    else:
        topk = k
    order = np.argsort(y_pred, kind="stable")[-topk:][::-1]
    y_true = y_true[order]

    # linear gain, not seansitive to position
    # gains = y_true
    # exponential gain, seansitive to position
    gains = 2 ** y_true - 1

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)

    # return DCG@k
    return np.sum(gains / discounts)

def nDCG_score(y_true, y_pred, k):
    best = DCG_score(y_true, y_true, k)
    real = DCG_score(y_true, y_pred, k)
    # return nDCG@k
    return real / best if best != 0 else 0

# %%
def evaluate(y_true, y_pred, k, length=50):
    NDCG, HR, total = 0.0, 0.0, 0

    for i in range(0, len(y_true), length):
        total += 1

        cur_y_true = y_true[i : i+length]
        cur_y_pred = y_pred[i : i+length]

        NDCG += nDCG_score(cur_y_true, cur_y_pred, k)
        HR   += HR_score  (cur_y_true, cur_y_pred, k)
    
    return NDCG / total, HR / total

In [19]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
x_test = torch.Tensor(df_test.vector)
y_test = torch.from_numpy(df_test.label.values.reshape(-1,1).astype(np.float32))
# 放在GPU中运算

x_test = x_test.to(device)
y_test = y_test.to(device)
model = model.to(device)

In [20]:
y_pred = model(x_test)
y_true = np.array(to_list(y_test))
y_p = np.array(to_list(y_pred))

In [21]:
for k in range(5,11):
    ndcg, hr = evaluate(y_true,y_p,k)
    print('k: ', k, '  ','NDCG: ',ndcg, 'HR: ', hr)

k:  5    NDCG:  0.5646329214299596 HR:  0.6697452003362074
k:  6    NDCG:  0.5803189363633087 HR:  0.7105670702658626
k:  7    NDCG:  0.5926999842568663 HR:  0.7438664149135485
k:  8    NDCG:  0.6028582812213075 HR:  0.7723349671000992
k:  9    NDCG:  0.6117881470374897 HR:  0.7983694728668717
k:  10    NDCG:  0.6188377182743429 HR:  0.8192095685325907


In [14]:
df_train = df_train.sort_index(ascending=True)
df_train

,author,paper,label
0,a678283,p782,1
1,a644976,p872,1
2,a107969,p872,1
3,a16959,p977,1
4,a500399,p1014,1
...,...,...,...
163067,a19813,p1311670,0
163068,a782515,p1531699,0
163069,a1456526,p1491086,0
163070,a688,p1693866,0


In [19]:
df_train.to_pickle('/home/duyongkang/PaperRec/Data/Train_1feature.pkl')

In [ ]:
df_test.to_pickle('home/duyongkang/PaperRec/Data/Glove_test.pkl')